# Import and Initialize

In [193]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

file_path = '/Users/haydnjones/Documents/GitHub/flight-crashes/Plane Crashes.csv'

df = pd.read_csv(file_path)
df = pd.DataFrame(df)

# Cleaning

In [196]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df.columns = pd.Series(df.columns).apply(lambda col: col.lower())
df.dropna(how='all', inplace= True)
df.fillna("unknown", inplace = True)
df.columns = df.columns.str.replace(' ', '_')

df['crew_on_board'] = df['crew_on_board'].replace('unknown', '0.0')
df['crew_on_board'] = pd.to_numeric(df['crew_on_board'], errors='coerce')

df['pax_on_board'] = df['pax_on_board'].replace('unknown', '0.0')
df['pax_on_board'] = pd.to_numeric(df['pax_on_board'], errors='coerce')

df['crew_fatalities'] = df['crew_fatalities'].replace('unknown', '0.0')
df['crew_fatalities'] = pd.to_numeric(df['crew_fatalities'], errors='coerce')

df['pax_fatalities'] = df['pax_fatalities'].replace('unknown', '0.0')
df['pax_fatalities'] = pd.to_numeric(df['pax_fatalities'], errors='coerce')

df['other_fatalities'] = df['other_fatalities'].replace('unknown', '0.0')
df['other_fatalities'] = pd.to_numeric(df['other_fatalities'], errors='coerce')

df['total_on_board'] = df['crew_on_board'] + df['pax_on_board']
df['total_fatalities'] = df['crew_fatalities'] + df['pax_fatalities'] + df['other_fatalities']

df_cleaned = df.drop(['crew_on_board', 'crew_fatalities', 'pax_on_board', 'pax_fatalities', 'other_fatalities', 'msn', 'yom', 'flight_no.'], axis=1)

order = [
    "date", "time", "aircraft", "operator", "flight_phase",
    "flight_type", "crash_site", "schedule", "crash_location",
    "country", "region", "circumstances", "crash_cause",
    "total_on_board", "total_fatalities", "survivors"
]

df_cleaned = df_cleaned[order]

In [198]:
passenger_types = [
    "scheduled revenue flight",
    "charter/taxi (non scheduled revenue flight)",
    "private",
    "executive/corporate/business",
    "ferry"
    "cargo",
    "positioning",
    "ambulance",
    "topographic",
    "geographical / geophysical / scientific",
    "illegal (smuggling)",
    "spraying (agricultural)",
    "humanitarian",
    "meteorological / weather",
    "fire fighting"
]

df_cleaned['flight_type'] = df_cleaned['flight_type'].apply(lambda x: 'passenger flights' if x in passenger_types else None)

df_cleaned = df_cleaned.dropna(subset=['flight_type']) 

In [200]:
threshold_year = 1950

df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
df_cleaned = df_cleaned[df_cleaned['date'].dt.year >= threshold_year]k

In [217]:
df_cleaned

,date,time,aircraft,operator,flight_phase,flight_type,crash_site,schedule,crash_location,country,region,circumstances,crash_cause,total_on_board,total_fatalities,survivors
9071,1950-01-03,unknown,avro 652 anson,united air services - tanzania,landing (descent or approach),passenger flights,"plain, valley",unknown,kazimzumbwi pwani region,tanzania,africa,"while approaching dar es-salaam, the twin engine aircraft went out of control, dove into the ground and crashed in a dense wooded area located in kazimzumbwi, about 30 km southwest of dar es-salaam. both crew members were killed.",unknown,2.0,2.0,no
9075,1950-01-07,unknown,boeing 247,lineas aéreas guerrero oaxaca - lagosa,takeoff (climb),passenger flights,"plain, valley",unknown,mexico city federal district of mexico city,mexico,central america,"few minutes after takeoff from mexico city, the twin engine aircraft crashed in unknown circumstances in los pajaritos, about 23 km north of the airfield. the occupant fate remains unknown.",unknown,0.0,0.0,unknown
9081,1950-01-14,unknown,de havilland dh.89 dragon rapide,new zealand national airways,parking,passenger flights,airport (less than 10 km from airport),rotorua – hamilton,rotorua bay of plenty regional council,new zealand,oceania,"the aircraft was parked at rotorua airport and ready for its schedule flight to hamilton airport (rukuhia). during engine start up, a fire erupted from the left engine and the fire spread to the dry grass. the aircraft caught fire and all five occupants were able to vacate the cabin before the aircraft would be completely destroyed by fire.",technical failure,5.0,0.0,yes
9082,1950-01-18,unknown,douglas dc-3,trans asiatic airlines - taa,takeoff (climb),passenger flights,airport (less than 10 km from airport),unknown,yangon yangon region,myanmar,asia,"an undercarriage failed during takeoff roll. the aircraft went out of control, veered off runway and came to rest. there were no casualties.",technical failure,0.0,0.0,yes
9086,1950-01-22,unknown,de havilland dh.104 dove,central african airways - caa,landing (descent or approach),passenger flights,airport (less than 10 km from airport),unknown,livingstone southern,zambia,africa,crashed on final approach to livingstone airport. the occupant fate remains unknown.,unknown,0.0,0.0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28529,2022-05-11,unknown,de havilland dhc-6 twin otter,caverton helicopters,flight,passenger flights,"plain, valley",yaoundé – dompta – belabo,nanga eboko centre,cameroon,africa,"the twin engine airplane departed yaoundé at 1246lt on a charter flight to dompta and belabo, carrying 9 employees of the cameroon oil transportation company (cotco) and two crew members. en route, the airplane crashed in unknown circumstances near the village of nanga eboko. all 11 occupants were killed.",unknown,11.0,11.0,no
28530,2022-05-12,8h 4m 0s,airbus a319,tibet airlines,takeoff (climb),passenger flights,airport (less than 10 km from airport),chongqing – nyingchi,chongqing-jiangbei sichuan,china,asia,"the airplane was departing chongqing-jiangbei airport on a regular schedule service to nyingchi, tibet, carrying 113 passengers and a crew of nine. during the takeoff run from runway 03, the captain encountered an unexpected situation and decided to abandon the takeoff procedure. he initiated an emergency braking manoeuvre when the airplane started to deviate to the left. it veered off runway, went through a grassy area, lost its undercarriage and both engines before coming to rest on a parallel taxiway, bursting into flames. 36 occupants were injured while all others evacuated safely. the aircraft was destroyed.",unknown,122.0,0.0,yes
28532,2022-05-23,18h 29m 0s,piper pa-61 aerostar (ted smith 601),raul ignacion posada,flight,passenger flights,"plain, valley",celaya - durango,durango durango,mexico,central america,"while approaching durango airport on a flight from celaya, the twin engine aircraft entered an uncontrolled desce

# Whatever I'm doing

In [211]:
import pandas as pd
pd.set_option('display.max_colwidth', None) 
from bs4 import BeautifulSoup
import requests
import pandas as pd




url = 'https://skytraxratings.com/a-z-of-airline-ratings'

response = requests.get(url)
soup = BeautifulSoup(response.content)   

airlines = []
for airline in soup.find_all('td', class_ = 'column-2'):
    airlines.append(airline.get_text())

ratings = []
for rating in soup.find_all('td', class_ = 'column-1'):
    ratings.append(rating.get_text())


df_airlines = pd.DataFrame({
    "airlines": airlines,      
    "ratings": ratings
})

# Keep for later maybe

In [156]:
df_takeoff_landing = df_cleaned

In [157]:
unwanted_phases = ['flight', 'unknown', 'parking', 'taxiing']
df_takeoff_landing = df_takeoff_landing[~df_takeoff_landing['flight_phase'].isin(unwanted_phases)]